# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = ''

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [10]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['employees', 'salaries', 'departments'],
        'definition': ['list of employees with names, department and other personal data like address and studies, and salaries', 'salaries per month for each employee', 'list of departments with respective employees']}
df = pd.DataFrame(data)
print(df)

         table                                         definition
0    employees  list of employees with names, department and o...
1     salaries               salaries per month for each employee
2  departments      list of departments with respective employees


In [11]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [12]:
print(text_tables)

employees: list of employees with names, department and other personal data like address and studies, and salaries
salaries: salaries per month for each employee
departments: list of departments with respective employees


In [13]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [15]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='what is the total spend with salaries for department A last month?')

In [ ]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "salary"]
}
```


In [16]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question='what is the most costly department in the company?')

In [17]:
print(return_OAI(pqt3))

```json
{
    "tables": ["salaries", "employees", "departments"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [21]:
# Table and definitions sample
authors_data = {'table': ['authors', 'titles', 'publisher', 'sales'],
        'definition': ['list of book authors with names and other personal data', 'titles of each author and year of publication', 'publisher of each book', 'sales of each book per month, quantities and sales income']}
dfa = pd.DataFrame(authors_data)
print(dfa)




       table                                         definition
0    authors  list of book authors with names and other pers...
1     titles      titles of each author and year of publication
2  publisher                             publisher of each book
3      sales  sales of each book per month, quantities and s...


In [24]:
text_tables_authors = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in dfa.iterrows()])

In [25]:
print(text_tables_authors)

authors: list of book authors with names and other personal data
titles: titles of each author and year of publication
publisher: publisher of each book
sales: sales of each book per month, quantities and sales income


In [26]:
prompt_question_tables_authors = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""

In [28]:
#Creating the prompt, with the user questions and the tables definitions.
pqt2 = prompt_question_tables_authors.format(tables=text_tables_authors, question='what is the top selling author and how many books did they sell?')

In [30]:
print(return_OAI(pqt2))

{
    "tables": ["authors", "titles", "sales"]
}


Learnings:
* it is clear that the tested model is good at handling the task at hand, even if it's not specialized.
* Learned how to define variables inside a prompt and how to access them when calling the api.